In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import pandas as pd
import gc
import numpy as np

In [2]:
starttime = time.time()
# read in data
mainFrame=pd.read_csv('20062007Small.csv',parse_dates=['date'])
#model = joblib.load(r'C:\Hedge Fund Project\training\modelv1.plk')
mainFrame.set_index(['entityID','date'],inplace=True)
#mainFrame=mainFrame20052018
mainFrame.sort_index(inplace=True)
targets=mainFrame.iloc[:,-7:]
features = mainFrame.iloc[:,:-7]
gc.collect()
endtime = time.time()
print("It takes {}s to load data".format(endtime-starttime))

It takes 55.18465185165405s to load data


In [3]:
maskTrain=(mainFrame.index.get_level_values(1)>='2006') & (mainFrame.index.get_level_values(1)<'2006-10-01')

maskTest=(mainFrame.index.get_level_values(1)>='2006-10-01') & (mainFrame.index.get_level_values(1)<'2007')

x_train=np.array(features[maskTrain])
y_train=np.array(targets['ztargetMedian5'][maskTrain])
x_train[np.isinf(x_train)]=100000000
y_train=y_train*1
y_train=y_train.astype(int)

x_test=np.array(features[maskTest])
y_test = np.array(targets['ztargetMedian5'][maskTest])
x_test[np.isinf(x_test)]=100000000
y_test=y_test*1
y_test=y_test.astype(int)

input_dims=x_train.shape[1]

In [4]:
# Parameters
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(input_dims,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='sigmoid'),
    layers.Dense(1)
])
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.binary_crossentropy,
             metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                12096     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 14,241
Trainable params: 14,241
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.fit(x_train, y_train, batch_size=50, epochs=50, validation_split=0.1, verbose=1)

Train on 442966 samples, validate on 49219 samples
Epoch 1/50
442966/442966 [==============================] - 58s 131us/sample - loss: 0.8033 - accuracy: 0.5052 - val_loss: 0.6921 - val_accuracy: 0.5156
Epoch 2/50
442966/442966 [==============================] - 45s 102us/sample - loss: 0.6939 - accuracy: 0.5075 - val_loss: 0.6921 - val_accuracy: 0.5216
Epoch 3/50
442966/442966 [==============================] - 33s 76us/sample - loss: 0.6940 - accuracy: 0.5091 - val_loss: 0.6926 - val_accuracy: 0.5126
Epoch 4/50
442966/442966 [==============================] - 30s 69us/sample - loss: 0.6937 - accuracy: 0.5114 - val_loss: 0.6930 - val_accuracy: 0.5164
Epoch 5/50
442966/442966 [==============================] - 31s 70us/sample - loss: 0.6936 - accuracy: 0.5105 - val_loss: 0.6916 - val_accuracy: 0.5200
Epoch 6/50
442966/442966 [==============================] - 31s 69us/sample - loss: 0.6936 - accuracy: 0.5097 - val_loss: 0.6924 - val_accuracy: 0.5223
Epoch 7/50
442966/442966 [=========

In [7]:
result = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['loss', 'accuracy']
[0.6932700236186291, 0.50387675]
